In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv('Datasets/UpdatedResumeDataSet[1].csv')

In [ ]:
print(data, '\n\n')

         Category                                             Resume
0    Data Science  Skills * Programming Languages: Python (pandas...
1    Data Science  Education Details \r\nMay 2013 to May 2017 B.E...
2    Data Science  Areas of Interest Deep Learning, Control Syste...
3    Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4    Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...
..            ...                                                ...
957       Testing  Computer Skills: â¢ Proficient in MS office (...
958       Testing  â Willingness to accept the challenges. â ...
959       Testing  PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960       Testing  COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961       Testing  Skill Set OS Windows XP/7/8/8.1/10 Database MY...

[962 rows x 2 columns] 




In [ ]:
print(data['Category'].unique())
print(len(data['Category'].unique()))

['Data Science' 'HR' 'Advocate' 'Arts' 'Web Designing'
 'Mechanical Engineer' 'Sales' 'Health and fitness' 'Civil Engineer'
 'Java Developer' 'Business Analyst' 'SAP Developer' 'Automation Testing'
 'Electrical Engineering' 'Operations Manager' 'Python Developer'
 'DevOps Engineer' 'Network Security Engineer' 'PMO' 'Database' 'Hadoop'
 'ETL Developer' 'DotNet Developer' 'Blockchain' 'Testing']
25


In [ ]:
# print(data['Resume'].unique())
print(len(data['Resume'].unique()))

166


In [ ]:
print(data.describe())

              Category                                             Resume
count              962                                                962
unique              25                                                166
top     Java Developer  Technical Skills Web Technologies: Angular JS,...
freq                84                                                 18


In [ ]:
print('The number of NaN','\n', np.sum(data.isnull()), '\n\n')

The number of NaN 
 Category    0
Resume      0
dtype: int64 




/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
print('features: ', data.columns, '\n\n')

features:  Index(['Category', 'Resume'], dtype='object') 




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data['Resume'], data['Category'], test_size=0.33, random_state=42)

In [ ]:
#Vectorizing with tf-idf
vectorizer = TfidfVectorizer(lowercase=True, analyzer='word')
X_train_tf_idf = vectorizer.fit_transform(X_train)
X_test_tf_idf = vectorizer.transform(X_test)
print(vectorizer.get_feature_names_out())
print(X_train_tf_idf.shape)

['000' '01' '017' ... 'zz' 'ã¼' 'ã¼â']
(644, 7513)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # y_train: list of string labels
y_test_encoded = label_encoder.transform(y_test)

clf = LogisticRegression(max_iter=500)
clf.fit(X_train_tf_idf, y_train_encoded)

preds = clf.predict(X_test_tf_idf)

acc = accuracy_score(y_test_encoded, preds)
print(f"Test Accuracy: {acc:.4f}")

print("\nClassification Report:")
print(classification_report(y_test_encoded, preds, target_names=label_encoder.classes_))

decoded_preds = label_encoder.inverse_transform(preds)
print("Example decoded predictions:", decoded_preds[:5])



Test Accuracy: 0.9937

Classification Report:
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         7
                     Arts       1.00      1.00      1.00        10
       Automation Testing       1.00      1.00      1.00         7
               Blockchain       1.00      1.00      1.00        12
         Business Analyst       1.00      1.00      1.00         9
           Civil Engineer       1.00      1.00      1.00        14
             Data Science       1.00      1.00      1.00        12
                 Database       1.00      1.00      1.00         9
          DevOps Engineer       1.00      0.90      0.95        21
         DotNet Developer       1.00      1.00      1.00        11
            ETL Developer       1.00      1.00      1.00         9
   Electrical Engineering       1.00      1.00      1.00        10
                       HR       1.00      1.00      1.00        18
               

In [ ]:
#text encoding with bert
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import torch

# Load BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Get CLS token
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: (1, 768)
    return cls_embedding.cpu().numpy().flatten()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
X_train_bert = np.array([get_bert_embedding(text) for text in tqdm(X_train)])
X_test_bert = np.array([get_bert_embedding(text) for text in tqdm(X_test)])

100%|██████████| 318/318 [00:19<00:00, 16.47it/s]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

logreg_bert = LogisticRegression(max_iter=1000)
logreg_bert.fit(X_train_bert, y_train_enc)

preds = logreg_bert.predict(X_test_bert)
print("BERT+LogReg Accuracy:", accuracy_score(y_test_enc, preds))


BERT+LogReg Accuracy: 0.9937106918238994


In [ ]:
torch.save(model.state_dict(), "mlp_model.pt")

In [ ]:
import joblib

# Save TF-IDF vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

# Save label encoder (for transforming labels to/from numbers)
joblib.dump(label_encoder, "label_encoder.pkl")

# Save logistic regression trained on TF-IDF features
joblib.dump(clf, "logreg_tfidf.pkl")

# Save logistic regression trained on BERT embeddings
joblib.dump(logreg_bert, "logreg_bert.pkl")

['logreg_bert.pkl']

In [ ]:
from google.colab import files
files.download("logistic_model.pkl")
files.download("tfidf_vectorizer.pkl")
files.download("logreg_bert.pkl")
files.download("label_encoder.pkl")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>